In [1]:
import os 
import logging 
import sys 
import openai 

from llama_index import (
    KnowledgeGraphIndex, 
    LLMPredictor, 
    ServiceContext, 
    SimpleDirectoryReader
)

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore
from llama_index.llms import OpenAI

os.environ["OPENAI_API_KEY"] = "sk-..."
openai.api_key = "sk-..."


llm = OpenAI(temperature = 0, model = "text-davinci-002")
service_context = ServiceContext.from_defaults(llm = llm, chunk_size_limit = 512)

In [ ]:
%pip install ipython-ngql nebula3-python

In [2]:
os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula"  
os.environ[
    "NEBULA_ADDRESS"
] = "127.0.0.1:9669"  

space_name = "sample_ccp"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  

tags = ["entity"]

In [3]:
graph_store = NebulaGraphStore(
    space_name = space_name,
    edge_types = edge_types,
    rel_prop_names = rel_prop_names,
    tags = tags,
)
storage_context = StorageContext.from_defaults(graph_store = graph_store)

In [4]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")
loader = WikipediaReader()
documents = loader.load_data(
    pages = ["Chinese Communist Party"], auto_suggest = False
)

In [5]:
kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context = storage_context,
    max_triplets_per_chunk = 10,
    service_context = service_context,
    space_name = space_name,
    edge_types = edge_types,
    rel_prop_names = rel_prop_names,
    tags = tags,
    include_embeddings = True,
)

In [6]:
kg_index

In [7]:
%pip install ipython-ngql networkx pyvis
%load_ext ngql
%ngql --address 127.0.0.1 --port 9669 --user root --password <password>

Note: you may need to restart the kernel to use updated packages.
Connection Pool Created


,Name
0,ccp
1,ccp_graph
2,llamaindex
3,sample_ccp


In [9]:
%ngql USE sample_ccp;
%ngql MATCH ()-[e]->() RETURN e LIMIT 10

,e
0,"(""Brown"")-[:relationship@3827542397268656921{r..."
1,"(""CCP"")-[:relationship@-9145588482981122771{re..."
2,"(""CCP"")-[:relationship@-8943842838841622963{re..."
3,"(""CCP"")-[:relationship@-8833295091973254590{re..."
4,"(""CCP"")-[:relationship@-8479060021137937403{re..."
5,"(""CCP"")-[:relationship@-7350560827441132359{re..."
6,"(""CCP"")-[:relationship@-7182510495877438698{re..."
7,"(""CCP"")-[:relationship@-7042246974363447575{re..."
8,"(""CCP"")-[:relationship@-6983053292402816223{re..."
9,"(""CCP"")-[:relationship@-6983053292402816223{re..."


In [10]:
%ng_draw

<class 'pyvis.network.Network'> |N|=12 |E|=10

In [11]:
from llama_index.query_engine import KnowledgeGraphQueryEngine
from IPython.display import Markdown, display
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore

query_engine = KnowledgeGraphQueryEngine(
    storage_context = storage_context,
    service_context = service_context,
    llm = llm,
    verbose = True,
)

In [12]:
response = query_engine.query(
    "Tell me about the Chinese Civil War?",
)
display(Markdown(f"<b>{response}</b>"))

Graph Store Query:
```
MATCH (c:`entity`)-[:`relationship`]->(w:`entity`) WHERE c.`entity`.`name` == 'Chinese Civil War'
RETURN w.`entity`.`name`;
```
Graph Store Response:
{'w.entity.name': []}
Final Response: 

The Chinese Civil War was a conflict that took place in China between 1927 and 1950. The war was fought between the Chinese Nationalists, led by Chiang Kai-shek, and the Chinese Communists, led by Mao Zedong. Ultimately, the Communists were victorious, and in 1949 they established the People's Republic of China.


<b>

The Chinese Civil War was a conflict that took place in China between 1927 and 1950. The war was fought between the Chinese Nationalists, led by Chiang Kai-shek, and the Chinese Communists, led by Mao Zedong. Ultimately, the Communists were victorious, and in 1949 they established the People's Republic of China.</b>

In [18]:
%ngql USE llamaindex; MATCH p=(n)-[e:relationship*1..2]-() WHERE id(n) in ['Mao', 'Chinese Civil War', 'Shuanggui'] RETURN p

,p
0,"(""Chinese Civil War"" :entity{name: ""Chinese Ci..."
1,"(""Chinese Civil War"" :entity{name: ""Chinese Ci..."
2,"(""Chinese Civil War"" :entity{name: ""Chinese Ci..."
3,"(""Chinese Civil War"" :entity{name: ""Chinese Ci..."
4,"(""Chinese Civil War"" :entity{name: ""Chinese Ci..."
...,...
200,"(""Shuanggui"" :entity{name: ""Shuanggui""})-[:rel..."
201,"(""Shuanggui"" :entity{name: ""Shuanggui""})-[:rel..."
202,"(""Shuanggui"" :entity{name: ""Shuanggui""})-[:rel..."
203,"(""Shuanggui"" :entity{name: ""Shuanggui""})-[:rel..."


In [19]:
%ng_draw

<class 'pyvis.network.Network'> |N|=110 |E|=399